# Datestamp Handling

This tutorial covers how `projio` automatically organizes files using date-based prefixes and directories.

## Why Datestamps?

When running experiments or processing data over time, organizing outputs by date helps:

- Find results from a specific day
- Avoid overwriting previous outputs
- Track the history of your work
- Compare results across different runs

## Basic Datestamp Usage

In [1]:
import tempfile
from project_io import ProjectIO

tmp = tempfile.mkdtemp()

# Enable datestamps
io = ProjectIO(root=tmp, use_datestamp=True, auto_create=False)

# Get today's datestamp
print(f"Today's datestamp: {io.datestamp_value()}")

Today's datestamp: 2025_12_23


## Datestamp Placement Options

You can control where datestamps appear using the `datestamp_in` parameter:

- `"dirs"`: Add datestamp as a directory level
- `"files"`: Add datestamp as a filename prefix
- `"both"`: Add datestamp to both directories and filenames
- `"none"`: Disable datestamps (same as `use_datestamp=False`)

### Datestamp in Directories

In [2]:
io_dirs = ProjectIO(
    root=tmp,
    use_datestamp=True,
    datestamp_in="dirs",
    auto_create=False
)

# Mock a fixed datestamp for reproducible output
io_dirs.datestamp_value = lambda ts=None: "2024_03_15"

path = io_dirs.path_for('outputs', 'results', ext='.csv')
print(f"Path with dir datestamp: {path}")
# Output: .../outputs/2024_03_15/results.csv

Path with dir datestamp: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/2024_03_15/results.csv


### Datestamp in Filenames

In [3]:
io_files = ProjectIO(
    root=tmp,
    use_datestamp=True,
    datestamp_in="files",
    auto_create=False
)

io_files.datestamp_value = lambda ts=None: "2024_03_15"

path = io_files.path_for('outputs', 'results', ext='.csv')
print(f"Path with file datestamp: {path}")
# Output: .../outputs/2024_03_15_results.csv

Path with file datestamp: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/2024_03_15__results.csv


### Datestamp in Both

In [4]:
io_both = ProjectIO(
    root=tmp,
    use_datestamp=True,
    datestamp_in="both",
    auto_create=False
)

io_both.datestamp_value = lambda ts=None: "2024_03_15"

path = io_both.path_for('outputs', 'results', ext='.csv')
print(f"Path with both datestamps: {path}")
# Output: .../outputs/2024_03_15/2024_03_15_results.csv

Path with both datestamps: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/2024_03_15/2024_03_15__results.csv


## Custom Datestamp Format

The default format is `%Y_%m_%d` (e.g., `2024_03_15`). You can customize this:

In [5]:
io_custom = ProjectIO(
    root=tmp,
    use_datestamp=True,
    datestamp_format="%Y%m%d",  # Compact format: 20240315
    auto_create=False
)

print(f"Custom format datestamp: {io_custom.datestamp_value()}")

Custom format datestamp: 20251223


## Parsing Datestamps

You can parse datestamps back to datetime objects:

In [6]:
# Parse a datestamp string using the instance method
# (uses the instance's datestamp_format automatically)
io = ProjectIO(root=tmp, use_datestamp=True, auto_create=False)
dt = io.parse_datestamp("2024_03_15")
print(f"Parsed datetime: {dt}")
print(f"Year: {dt.year}, Month: {dt.month}, Day: {dt.day}")

Parsed datetime: 2024-03-15 00:00:00
Year: 2024, Month: 3, Day: 15


## Overriding Datestamp Per-Call

You can override datestamp behavior for individual path calls:

In [7]:
io = ProjectIO(root=tmp, use_datestamp=True, datestamp_in="dirs", auto_create=False)
io.datestamp_value = lambda ts=None: "2024_03_15"

# Default behavior - uses datestamp
path_with = io.path_for('outputs', 'results', ext='.csv')
print(f"With datestamp: {path_with}")

# Override to skip datestamp for this call
path_without = io.path_for('outputs', 'results', ext='.csv', datestamp=False)
print(f"Without datestamp: {path_without}")

With datestamp: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/2024_03_15/results.csv
Without datestamp: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/results.csv


## Lightning Paths with Datestamps

Datestamps also work with Lightning-specific paths:

In [8]:
io = ProjectIO(root=tmp, use_datestamp=True, datestamp_in="dirs", auto_create=False)
io.datestamp_value = lambda ts=None: "2024_03_15"

# Checkpoint path with datestamp
ckpt = io.checkpoint_path('model', run='exp1')
print(f"Checkpoint: {ckpt}")

# TensorBoard run directory
tb_dir = io.tensorboard_run(run='exp1')
print(f"TensorBoard: {tb_dir}")

Checkpoint: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/lightning/checkpoints/2024_03_15/exp1/model.ckpt
TensorBoard: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/lightning/tensorboard/2024_03_15/exp1


## Temporary Override with Context Manager

Use the `using()` context manager to temporarily change datestamp behavior:

In [9]:
io = ProjectIO(root=tmp, use_datestamp=True, auto_create=False)
io.datestamp_value = lambda ts=None: "2024_03_15"

print(f"Datestamp enabled: {io.use_datestamp}")

with io.using(use_datestamp=False):
    print(f"Inside context: {io.use_datestamp}")
    path = io.path_for('outputs', 'test', ext='.txt')
    print(f"Path in context: {path}")

print(f"After context: {io.use_datestamp}")

Datestamp enabled: True
Inside context: False
Path in context: /private/var/folders/f7/7pcpvrhn0p9gw509gyzmh8fxrwyskv/T/tmpnvzeftah/test.txt
After context: True


## Best Practices

1. **Use `datestamp_in="dirs"`** for most cases - it keeps filenames clean and makes browsing by date easy

2. **Use `datestamp_in="files"`** when you need all outputs in a single directory but want to distinguish by date

3. **Override with `datestamp=False`** for files that shouldn't be date-organized (e.g., config files, README)

4. **Consider the format** - `%Y_%m_%d` sorts chronologically; `%d_%m_%Y` does not

## Next Steps

- Learn about [Lightning integration](03_lightning.ipynb) for ML workflows
- Explore [templates](04_templates.ipynb) for common file patterns
- Check out [advanced features](05_advanced.ipynb) like producer tracking